# An Introduction to Pandas Part II
### Harsha Kankanamge

In [5]:
import pandas as pd
pd.__version__

'0.23.4'

In [1]:
 # get working (current) directory
import os
os.getcwd()

'C:\\Users\\harsh\\Desktop\\GEG584\\Assignment_3'

In [6]:
filename = 'PlacePop_poverty2017.csv'
pop_df = pd.read_csv(filename,encoding='UTF-8')
pop_df.shape # How many rows and columns in the dataframe

(29323, 7)

In [ ]:
pop_df.head()

In [ ]:
pop_df.columns

In [ ]:
fields=[]
for x in [2,5,6]:fields.append(pop_df.columns[x])
fields

In [ ]:
pop_df = pd.read_csv(filename,usecols=fields)
pop_df.shape

### Find places similar in size to Buffalo, NY.
Calculate the z-score comparing the under 18 population of all places to the under 18 population of Buffalo, NY.                
Use this information to retain places with a under 18 population that is not significantly different from Buffalo's.                   

1.Get Buffalo's population (Under18Est and Under18Moe)                                
2.Calculate z-score comparing Buffalo to all other places

In [ ]:
bufEst = pop_df.loc[pop_df.Geography=='Buffalo city, New York',['Under18Est']].values[0,0]
bufMoe = pop_df.loc[pop_df.Geography=='Buffalo city, New York',['Under18Moe']].values[0,0]
bufEst,bufMoe

In [ ]:
pop_df['zBflo']=(pop_df['Under18Est']-bufEst)/((pop_df['Under18Moe']/1.645)**2+(bufMoe/1.645)**2)**.5
pop_df['zBflo'][:10]

In [ ]:
pop_df1=pop_df.loc[(pop_df['zBflo'].abs() < 1.645)].copy() # under 18 population same as Buffalo, 90% confidence

In [ ]:
pop_df1

### Result
This new dataframe contains under 18 population estimates for 7 places similar in size to Buffalo. We are interested in how Buffalo compares to these cities with respect to the percent of the under 18 population below the poverty level.                         

Since we only need the data for these 7 places, join the poverty rate data to this dataframe.

Analyze the results for these places.

In [ ]:
filename = 'poverty2017_5yr.csv'
fields = ['Geography', 'Under 18 BelPov Est','Under 18 BelPov MOE' ]
df = pd.read_csv(filename,usecols=fields)
df.columns=["Geography", "est","moe"]
df.columns

Since the population dataframe and the poverty rate dataframe have a common field (Geography), a simple merge statement suffices.

In [ ]:
mergedf = pd.merge(pop_df1,df)
mergedf

In [ ]:
mergedf['est']=mergedf['est'].apply(lambda x: pd.to_numeric(x, errors='coerce'))
mergedf['moe']=mergedf['moe'].apply(lambda x: pd.to_numeric(x, errors='coerce'))

In [ ]:
mergedf.dropna(inplace=True)
mergedf

## Calculate z-score

In [ ]:
bufEst = mergedf.loc[mergedf.Geography=='Buffalo city, New York']
bufEst

In [ ]:
bufEst = mergedf.loc[mergedf.Geography=='Buffalo city, New York',['est']].values[0,0]
bufMoe = mergedf.loc[mergedf.Geography=='Buffalo city, New York',['moe']].values[0,0]
bufEst,bufMoe

In [ ]:
mergedf['zBflo2']=(mergedf['est']-bufEst)/((mergedf['moe']/1.645)**2+(bufMoe/1.645)**2)**.5
mergedf['zBflo2'][:10]

In [ ]:
mergedf.loc[mergedf.zBflo2 > 1.645,['Geography','est','moe']] # percent poverty greater than Buffalo, 90% confidence

### Result
Only 1 Place in the U.S. similar in size to Buffalo has a higher proportion of population below the poverty level.

How many similarly sized places have a poverty rate that is not significantly different from Buffalo's?

In [ ]:
mergedf['zBflo2']=mergedf['zBflo2'].abs()

In [ ]:
mergedf.loc[mergedf.zBflo2 <= 1.645,['Geography','est','moe']] # percent poverty similar to Buffalo, 90% confidence

### Result Only 3!
How many places with under 18 population greater than 80000 have higher poverty rates than Buffalo?

In [ ]:
bigdf=pop_df.loc[(pop_df.zUB >= 1.645)].copy() # under 18 population greater than or equal to 80000, 90% confidence
bigdf.shape

In [ ]:
bigMergedf = pd.merge(bigdf,df)
bigMergedf.tail()

In [ ]:
bigMergedf['est']=bigMergedf['est'].apply(lambda x: pd.to_numeric(x, errors='coerce'))
bigMergedf['moe']=bigMergedf['moe'].apply(lambda x: pd.to_numeric(x, errors='coerce'))

In [ ]:
bigMergedf['zBflo2']=(bigMergedf['est']-bufEst)/((bigMergedf['moe']/1.645)**2+(bufMoe/1.645)**2)**.5
bigMergedf['zBflo2'][:10]

In [ ]:
bigMergedf.loc[bigMergedf.zBflo2 > 1.645,['Geography','est','moe']] # percent poverty greater than Buffalo, 90% confidence

In [ ]:
bigMergedf['zBflo2']=bigMergedf['zBflo2'].abs()
bigMergedf.loc[bigMergedf.zBflo2 <= 1.645,['Geography','est','moe']] # percent poverty similar to Buffalo, 90% confidence

### Summary
Fifty four large places (Mesa city,Phoenix city, Arizona , Anaheim city, California and more) and 1 similar sized place (Rochester, NY) have under 18 (childrens) poverty rates higher than Buffalo's. Three similary sized places (Birmingham city, Alabama, Syracuse city, New York and Dayton city, Ohio) have poverty rates that are not significantly different from Buffalo's for under 18 (childrens).



# Challenge:

Instead of using total population as the measure of size for the comparisons, use the proportion of children in the population. These values can be calculated from the PlacePop_poverty2017.csv file.

In [ ]:
import pandas as pd
df_2=pd.read_csv('PlacePop_poverty2017.csv')

In [ ]:
df_2[:5]

 ### Calculate the estimated proportion of children PctKidEst = (Under18Est/PopEst) and the margin of error

In [ ]:
df_2['PctKidEst'] = (df_2['Under18Est']/df_2['PopEst']) 

### PctKidMoe = ((1/PopEst) * sqrt( Under18Moe^2 - PctKidEst^2 * PopMoe^2) *1.645

In [ ]:
df_2['PctKidMoe'] = ((1/df_2['PopEst']) * ( df_2['Under18Moe']**2 - (df_2['PctKidEst']**2 * df_2['PopMoe']**2))**0.5)

In [ ]:
df_2[:5]

In [ ]:
df_2['PctKidEst']=df_2['PctKidEst']*100
df_2['PctKidMoe']=df_2['PctKidMoe']*100

In [ ]:
df_2=df_2[["Geography", "PctKidEst","PctKidMoe"]]
df_2[:5]

### Find places similar in size to Buffalo, NY.
Calculate the z-score comparing the under 18 population of all places to the under 18 population of Buffalo, NY.                
Use this information to retain places with a under 18 population that is not significantly different from Buffalo's.                   

1.Get Buffalo's population (Under18Est and Under18Moe)                                
2.Calculate z-score comparing Buffalo to all other places

In [ ]:
bufEst = df_2.loc[df_2.Geography=='Buffalo city, New York',['PctKidEst']].values[0,0]
bufMoe = df_2.loc[df_2.Geography=='Buffalo city, New York',['PctKidMoe']].values[0,0]
bufEst,bufMoe

In [ ]:
df_2['zBflo2']=(df_2['PctKidEst']-bufEst)/((df_2['PctKidMoe']/1.645)**2+(bufMoe/1.645)**2)**.5
df_2['zBflo2'][:10]

In [ ]:
df_2.loc[(df_2['zBflo2'].abs() < 1.645)] # percent under 18 same as Buffalo, 90% confidence

## Result
18804 places have percent population under 18 equal to Buffalo'.   
Comapre to poverty rate for under 18 in Buffalo to the rate for these places 

## Find all places with PctKidEst similar to Buffalo's (extra)

In [ ]:
bufEst1 = df_2.loc[df_2.Geography=='Buffalo city, New York',['PctKidEst']].values[0,0]
bufEst1

In [ ]:
PctKidEs_equ_buff=df_2.loc[df_2.PctKidEst==bufEst1]
PctKidEs_equ_buff

## Round up and down to the nearest tens around Buffalo's

In [ ]:
PctKidEs_equ_buff1=df_2[(df_2['PctKidEst']<30)&(df_2['PctKidEst']>20)]
PctKidEs_equ_buff1